# 過去一週間の世界の地震の可視化

## 概要
本課題では、`pandas`と`folium`ライブラリを使用し、USGS（アメリカ地質調査所）から取得した過去一週間の世界の地震データを分析・可視化しました。

## 作品の詳細
1.  **データ**: USGSのGeoJSONフィードから最新の地震データを取得しました。
2.  **時間設定**: 地震発生時刻をUTCから日本時間（JST）に変換し、日付を変更しました。
3.  **地図**: `folium`を使用し、以下の条件で各地震を地図上にプロットしました。
    * **色**: マグニチュードに応じて緑（M<4.0）、黄色（4.0≦M＜6.0）、赤（M≧6.0）に色分け。
    * **サイズ**: マグニチュードに応じてポイントの大きさを変更。
    * **情報表示**: ポイントにマウスをおくことで「場所、マグニチュード、日本日時、深さ」を表示。
4.  **画面表示**: 地図を開いたとき、データは全世界のものですが、**日本列島を中心にズームイン**した状態から開始するように設定しました。

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import requests
import sys

# --------------------------------------------------------------------------------

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson"
df = None 

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    data = response.json()

    earthquake_list = []
    for feature in data.get('features', []):
        props = feature.get('properties', {})
        geom = feature.get('geometry', {})
        
        mag = props.get('mag')
        place = props.get('place')
        time_ms = props.get('time')
        
        coordinates = geom.get('coordinates', [None, None, None])
        longitude = coordinates[0]
        latitude = coordinates[1]
        depth = coordinates[2]
        
        earthquake_list.append({
            'magnitude': mag, 'place': place, 'time_ms': time_ms,
            'latitude': latitude, 'longitude': longitude, 'depth': depth
        })

    df = pd.DataFrame(earthquake_list)

    if 'time_ms' in df.columns and not df['time_ms'].isnull().all():
        df['datetime_utc'] = pd.to_datetime(df['time_ms'], unit='ms', errors='coerce')
        df['datetime_jst'] = df['datetime_utc'] + pd.Timedelta(hours=9)
        df['date_jst'] = df['datetime_jst'].dt.strftime('%Y-%m-%d %H:%M:%S JST')
    else:
        df['date_jst'] = '日時不明'

    df.dropna(subset=['magnitude', 'latitude', 'longitude'], inplace=True)
    
    if df.empty:
        print("データが取得できませんでした。処理を終了します。")
        df = None

except Exception as e:
    print(f"データ処理中にエラーが発生しました: {e}", file=sys.stderr)
    df = None

    
# --------------------------------------------------------------------------------

if df is not None:
    
    def get_marker_color(magnitude):
        if magnitude >= 6.0: return 'red'
        elif magnitude >= 4.0: return 'yellow'
        else: return 'green'

    def get_marker_radius(magnitude):
        if magnitude >= 6.0: return 9
        elif magnitude >= 4.0: return 6
        else: return 3

    initial_location = [38, 138] 
    m = folium.Map(location=initial_location, zoom_start=6) 

    marker_cluster = MarkerCluster().add_to(m)

    for index, row in df.iterrows():
        mag = row['magnitude']
        lat = row['latitude']
        lon = row['longitude']
        
        popup_html = f"""
        <b>場所:</b> {row.get('place', '不明')}<br>
        <b>マグニチュード:</b> M{mag:.2f}<br>
        <b>日時 (JST):</b> {row.get('date_jst', '不明')}<br>
        <b>深さ:</b> {row.get('depth', '不明'):.2f} km
        """
        
        color = get_marker_color(mag)
        radius = get_marker_radius(mag)
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=radius,
            color='black',
            weight=0.5,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300), 
            tooltip=popup_html
        ).add_to(marker_cluster)

    map_file = "worldwide_earthquakes_japan_center.html"
    m.save(map_file)
    print(f"地図は '{map_file}' に保存されました。")
    
    m

地図は 'worldwide_earthquakes_japan_center.html' に保存されました。


この地図から分かることとして、地震活動が環太平洋火山帯に集中していることが挙がります。マーカーの色と大きさから、日本で観測された地震の大半はマグニチュード4.0未満の小規模（緑色）であり、M6.0以上の大規模な地震は非常に稀（赤色）であることが視覚的にわかります。また、地震の数が一週間の間で11回ほどあったことから、気づかないだけで地震が起きていることが分かります。次は、日本の地震の情報をメインにして地図が作れると良いと思いました。